In [1]:
# 모델 구축
# 입력값 토큰화, 임베딩
# 훈련을 정의하고 학습
# 평가
# 예측하기 위한 데이터를 토큰화, 임베딩후 predict
# 라벨 출력

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate, TimeDistributed, Layer

class SimpleAttention(Layer):
    def __init__(self):
        super(SimpleAttention, self).__init__()

    def call(self, encoder_outputs, decoder_output):
        # encoder_outputs: (batch_size, enc_timesteps, units)
        # decoder_output: (batch_size, 1, units)
        # 어텐션 스코어 계산 (batch_size, 1, enc_timesteps)
        score = tf.matmul(decoder_output, encoder_outputs, transpose_b=True)
        # 어텐션 가중치 계산 (batch_size, 1, enc_timesteps)
        attention_weights = tf.nn.softmax(score, axis=-1)
        # 컨텍스트 벡터 계산 (batch_size, 1, units)
        context_vector = tf.matmul(attention_weights, encoder_outputs)
        return context_vector

# 인코더 입력 정의: 영문 단어가 입력, 각 단어는 4자 길이, 모든 문자의 수는 171개
enc_IN = Input(shape=(4, 171))

# 인코더 LSTM 정의: 모든 타임스텝의 출력을 반환하도록 설정
encoder_lstm = LSTM(128, return_sequences=True, return_state=True)
enc_OUT, state_h, state_c = encoder_lstm(enc_IN)

# 디코더 입력 정의
dec_IN = Input(shape=(3, 171))  # 한글 단어는 2자 길이 + <Start> 토큰, 모든 문자의 수는 171개

# 디코더 LSTM 정의: 모든 시퀀스의 출력을 반환하도록 설정
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
dec_Y, _, _ = decoder_lstm(dec_IN, initial_state=[state_h, state_c])

# 어텐션 메커니즘 적용
attention_layer = SimpleAttention()

# 각 디코더 타임스텝에 대해 어텐션 적용
context_vectors = []
for t in range(dec_Y.shape[1]):
    context_vector = attention_layer(enc_OUT, tf.expand_dims(dec_Y[:, t, :], 1))
    context_vectors.append(context_vector)

# 컨텍스트 벡터를 시퀀스 형태로 결합
context_vectors = tf.concat(context_vectors, axis=1)

# 디코더 출력과 어텐션 컨텍스트 벡터 결합
decoder_combined_context = Concatenate(axis=-1)([dec_Y, context_vectors])

# 디코더 출력층 정의: 출력 크기는 모든 문자의 수인 171, softmax 활성화 함수를 사용
dec_OUT = TimeDistributed(Dense(171, activation='softmax'))(decoder_combined_context)

# 모델 정의: 인코더 입력(enc_IN)과 디코더 입력(dec_IN)을 모델의 입력으로, 디코더 출력을 모델의 출력으로 설정
model = Model(inputs=[enc_IN, dec_IN], outputs=[dec_OUT])

# 모델 요약 출력
model.summary()


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [3]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam') # metrics가 없으면 훈련시 loss만 출력, predict()를 사용 못함

In [4]:
import pandas as pd
import numpy as np
# <START>, <END>, <PAD>
arr1 = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
arr2 = pd.read_csv('korean.csv', header=None)
# print(arr2[0].values.tolist())
num_to_char = arr1 + arr2[0].values.tolist()
print(len(num_to_char))

171


In [5]:
char_to_num = {char:i for i, char in enumerate(num_to_char)}
# print(char_to_num)

In [6]:
raw = pd.read_csv('translate.csv', header=None)
eng_kor = raw.values.tolist()
print(len(eng_kor))

110


In [7]:
temp_eng = 'love'
temp_eng_n = [char_to_num[c] for c in temp_eng]
print(temp_eng_n)
temp_kor = '사랑'
np.eye(171)[temp_eng_n].shape

[14, 17, 24, 7]


(4, 171)

In [8]:
def encode(eng_kor):
    enc_in = []
    dec_in = []
    rnn_out = [] # decoder output
    for seq in eng_kor:
        eng = [char_to_num[c] for c in seq[0]]
        enc_in.append(np.eye(171)[eng])

        kor = [char_to_num[c] for c in ('S'+seq[1])]
        dec_in.append(np.eye(171)[kor])

        target = [char_to_num[c] for c in (seq[1] + 'E')]
        rnn_out.append(target)

    enc_in = np.array(enc_in)
    dec_in = np.array(dec_in)
    rnn_out = np.array(rnn_out)
    rnn_out = np.expand_dims(rnn_out, axis=2)
    return enc_in, dec_in, rnn_out

In [9]:
sample = [['word', '단어']]
encode(sample)[2]

array([[[ 61],
        [114],
        [  1]]])

In [10]:
X_enc, X_dec, y_rnn = encode(eng_kor)

In [11]:
%%time
model.fit([X_enc, X_dec], y_rnn, epochs=500)

Epoch 1/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 5.1362
Epoch 2/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 5.1011
Epoch 3/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 5.0539
Epoch 4/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 4.9821 
Epoch 5/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 4.8409
Epoch 6/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 4.5471 
Epoch 7/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 3.9557
Epoch 8/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 3.4860
Epoch 9/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.6136
Epoch 10/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.4760
Epoch 11/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 3.3666
Epoch 12/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 3.3876 
Epoch 13/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 3.3507
Epoch 14/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.2979
Epoch 15/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 3.2622
Epoch 16/500
4/4 ━━

In [12]:
enc_in, dec_in, _ = encode([['tall', 'PP']])
# print(enc_in)
pred = model.predict([enc_in, dec_in])
# print(pred.shape)
word = np.argmax(pred[0], axis=-1)
# print(word)
num_to_char[word[0]], num_to_char[word[1]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step


('크', '다')

In [13]:
from numpy.random import randint
pick = randint(0, len(eng_kor), 5)
choose = [ [eng_kor[i][0], 'PP'] for i in pick]
print(choose)

[['time', 'PP'], ['slow', 'PP'], ['once', 'PP'], ['word', 'PP'], ['slow', 'PP']]


In [14]:
enc_in, dec_in, _ = encode(choose)
pred = model.predict([enc_in, dec_in])
print(pred.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step
(5, 3, 171)


In [15]:
for i in range(len(choose)):
    eng = choose[i][0]
    word = np.argmax(pred[i], axis=-1)
    kor = ''
    for j in range(2):
        kor = kor + num_to_char[word[j]]
    print(eng, kor)

time 시간
slow 늦다
once 한번
word 단어
slow 늦다
